# Covid-19 Vaccines in California and their Impacts

## Data Analysis

### Hypothesis:

#### 1. College towns had an impact on Covid-19 surges in 2020.

#### 2. Vaccines have directly impacted California’s case and death rates.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as sts
import gmaps
import datetime as dt

#import API key for gmaps
from config import gkey
gmaps.configure(api_key=gkey)

In [ ]:
# import required dataframes:

clean_case = pd.read_csv('Data/clean_case.csv')
vaccine_df = pd.read_csv('Data/vaccine_df.csv')
ca_vaccinations = pd.read_csv('Data/ca_vaccinations.csv')
county_data = pd.read_csv('Data/county_data.csv')

#reformat dates:
clean_case['date']=pd.to_datetime(clean_case.date)
clean_case.sort_values(by='date')

ca_vaccinations['administered_date']=pd.to_datetime(ca_vaccinations.administered_date)
ca_vaccinations = ca_vaccinations.sort_values(by='administered_date')

## Research Analysis

### 1. How has the introduction of the Covid vaccine affected case/death rate in California?

In [ ]:
#Covid cases vs. vaccinations

#covid case & dates for bar graph
x1=(clean_case['date'])
y1=(clean_case['cases'])

#vaccine counts and dates for line graph
x2=ca_vaccinations['administered_date']
y2=ca_vaccinations['cumulative_doses_by_10000']

#format figure
fig, axis1 = plt.subplots(figsize=(12,8))
axis2=axis1.twinx()    #create a second axis sharing the x-axis

#create bar graph using CA case counts
axis1.bar(x1,y1,)
axis1.set_ylabel("CA Case Count")
axis1.set_xlabel('Date')

#plot cumulative vaccine counts on 2nd y-axis
axis2.plot(x2,y2,color='red')
axis2.set_ylabel('Cumulative CA Vaccine Doses \n(per 10,000)')

#change y-ticks for 2nd axis
axis2.set_yticks(np.arange(0,2000,500))
plt.title('Covid19 Case Count vs. Vaccine Doses \n California')
plt.savefig('Covid19_CaseCount_vs_Vaccine')
plt.show()

In [ ]:
#covid deaths vs. vaccinations 
death_by_date = clean_case.groupby(['date'],as_index=False).agg({'deaths':['sum']})
death_by_date.columns = death_by_date.columns.droplevel(1)

#covid case & dates for bar graph
x1=(death_by_date['date'])
y1=(death_by_date['deaths'])

#vaccine counts and dates for line graph
x2=ca_vaccinations['administered_date']
y2=ca_vaccinations['cumulative_doses_by_10000']

#format figure
fig, axis1 = plt.subplots(figsize=(12,8))
axis2=axis1.twinx()    #create a second axis sharing the x-axis

#create bar graph using CA case counts
axis1.bar(x1,y1,)
axis1.set_ylabel("CA Deaths")
axis1.set_xlabel('Date')

#plot cumulative vaccine counts on 2nd y-axis
axis2.plot(x2,y2,color='red')
axis2.set_ylabel('Cumulative CA Vaccine Doses \n(per 10,000)')

#change y-ticks for 2nd axis
axis2.set_yticks(np.arange(0,2000,500))

plt.title('Covid19 Death Count vs. Vaccine Doses \n California')
plt.savefig('Covid19_DeathCount_vs_Vaccine')

plt.show()

### 2. Do “College” towns play a larger role in case surges?

In [ ]:
#Grabbing 3rd quartile for heatmap max intensities
county_data.quantile(0.75)

# Locating all counties in the top quartile as "College Counties"
college_counties = county_data.loc[county_data['% student population']>=7.425]
len(college_counties)

In [ ]:
# Creating heat map of Covid-19 cases

center = 36.7783,-119.4179
counties = county_data[['latitude', 'longitude']]
cases = county_data['cases per 100,000']

fig1 = gmaps.figure(center=center, zoom_level=5.5)

heatmap_layer_cases = gmaps.heatmap_layer(counties, weights=cases, dissipating=False)
heatmap_layer_cases.max_intensity = 9283
heatmap_layer_cases.point_radius = 0.25

fig1.add_layer(heatmap_layer_cases)

#add college counties to the 2020 heatmap

info_box_template = """
<dl>
<dt>County</dt><dd>{county}</dd>
<dt>% Students</dt><dd>{% student population}</dd>
<dt>Cases per 100,000</dt><dd>{cases per 100,000}</dd>
</dl>
"""
# Store the DataFrame Row

county_info = [info_box_template.format(**row) for index, row in college_counties.iterrows()]
locations = college_counties[['latitude', 'longitude']]

# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content=county_info)
fig1.add_layer(markers)

# Display figure

fig1

The above heatmap was originally broken down by year to see the affect of lower student populations due to virtual learning. However, the maps were identical. This also suggests that each county is maintaining it's own Covid-19 curve. No county appears to have dramatically changed its numbers between 2020 and 2021.

In [ ]:
# Creating heat map of Vaccines

center = 36.7783,-119.4179
counties = county_data[['latitude', 'longitude']]
cases = county_data['Fully Vaccinated per 100,000']

fig3 = gmaps.figure(center=center, zoom_level=5.5)

heatmap_layer_vaccines = gmaps.heatmap_layer(counties, weights=cases, dissipating=False)
heatmap_layer_vaccines.max_intensity = 30233
heatmap_layer_vaccines.point_radius = 0.25
heatmap_layer_vaccines.gradient = [(255,0,0, 0),
                                   (255,0,0, 1),
                                   (255,255,0, 1),
                                   (0,255,0, 1)]

fig3.add_layer(heatmap_layer_vaccines)

#add college counties to the Vaccine heatmap

info_box_template = """
<dl>
<dt>County</dt><dd>{county}</dd>
<dt>% Students</dt><dd>{% student population}</dd>
<dt>Cases per 100,000</dt><dd>{Fully Vaccinated per 100,000}</dd>
</dl>
"""
# Store the DataFrame Row

county_info = [info_box_template.format(**row) for index, row in college_counties.iterrows()]
locations = college_counties[['latitude', 'longitude']]

# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content=county_info)
fig3.add_layer(markers)

# Display figure

fig3

Inverted colors on Vaccines heat map as a greater number of vaccines per 100,000 people is considered good.

### 3. Are there “Vaccination Deserts”? What impact have Covid-19 and the vaccines had in these areas?

In [ ]:
# find county population to normalize heat to show percent of population vaccinated
fig4 = gmaps.figure()
heatmap = gmaps.heatmap_layer(county_data[['latitude', 'longitude']],
                              weights=county_data['Fully Vaccinated per 100,000'],
                              max_intensity=50000,
                              dissipating=False,
                              point_radius=0.4)

info_box_template = """
<dl>
<dt>County</dt><dd>{county}</dd>
<dt>Fully Vaccinated per 100,000</dt><dd>{Fully Vaccinated per 100,000}</dd>
<dt>As of</dt><dd>03-31-2021
</dl>
"""
vaccinated_info = [info_box_template.format(**county) for county in county_data.to_dict(orient='records')]

symbols = gmaps.symbol_layer(county_data[['latitude', 'longitude']],
                             hover_text=county_data.county,
                             fill_opacity=0,
                             stroke_opacity=0,
                             info_box_content=vaccinated_info)
fig4.add_layer(symbols)
fig4.add_layer(heatmap)
fig4

In [ ]:
# quick summary of vaccination rates
vax_mean = np.mean(county_data['Fully Vaccinated per 100,000'])
vax_med = np.median(county_data['Fully Vaccinated per 100,000'])
vax_max = county_data['Fully Vaccinated per 100,000'].max()
vax_min = county_data['Fully Vaccinated per 100,000'].min()

print(f"As of 03-31-2021, the mean vaccination rate of California counties is {round(vax_mean,2)} people per 100,000, while the median is {vax_med}.")
print(f"The most vaccinated county is {list(county_data.loc[county_data['Fully Vaccinated per 100,000']==vax_max].county)[0]} County, with a vaccinated rate of {vax_max} people per 100,000.")
print(f"The least vaccinated county is {list(county_data.loc[county_data['Fully Vaccinated per 100,000']==vax_min].county)[0]} County, with a vaccinated rate of {vax_min} people per 100,000.")

In [ ]:
# quartile information
quartiles = county_data['Fully Vaccinated per 100,000'].quantile([0.25,0.5,0.75])
lowerq = quartiles[.25]
upperq = quartiles[.75]
iqr = upperq-lowerq

upper_bound = upperq + (1.5*iqr)
lower_bound = lowerq - (1.5*iqr)

In [ ]:
# find counties in the upper/lower 25% of vaccination rates
good_counties = county_data[county_data['Fully Vaccinated per 100,000']>=upperq].sort_values('Fully Vaccinated per 100,000',ascending=False).reset_index(drop=True)
bad_counties = county_data[county_data['Fully Vaccinated per 100,000']<=lowerq].sort_values('Fully Vaccinated per 100,000').reset_index(drop=True)
bad_counties = bad_counties[['county','latitude','longitude','Fully Vaccinated per 100,000']]
good_counties = good_counties[['county','latitude','longitude','Fully Vaccinated per 100,000']]

In [ ]:
# map out to see distribution of these on the map
fig5 = gmaps.figure()

info_box_template = """
<dl>
<dt>County</dt><dd>{county}</dd>
<dt>Fully Vaccinated per 100,000</dt><dd>{Fully Vaccinated per 100,000}</dd>
<dt>As of</dt><dd>03-31-2021
</dl>
"""
good_vax_info = [info_box_template.format(**county) for county in good_counties.to_dict(orient='records')]
bad_vax_info = [info_box_template.format(**county) for county in bad_counties.to_dict(orient='records')]

good_symbols = gmaps.symbol_layer(good_counties[['latitude', 'longitude']],
                             hover_text=good_counties.county,
                             fill_opacity=1,
                             stroke_opacity=0,
                             fill_color='blue',
                             info_box_content=good_vax_info)

bad_symbols = gmaps.symbol_layer(bad_counties[['latitude', 'longitude']],
                             hover_text=bad_counties.county,
                             fill_opacity=1,
                             stroke_opacity=0,
                             fill_color='red',
                             info_box_content=bad_vax_info)

fig5.add_layer(good_symbols)
fig5.add_layer(bad_symbols)
fig5

The map above shows the distribution of counties in the top 25% (blue) of vaccination rates and the bottom 25% (red) of vaccination rates.

### 4. Can we predict when California will return to “normal”?